In [14]:
#!/venv/bin/python
from hume import HumeBatchClient
from hume.models.config import FaceConfig
from hume.models.config import ProsodyConfig
import pandas as pd
import numpy as np
import json 

client = HumeBatchClient("GB5GR8utyBkDmLHy5Kh7mtAGGebtMK5R9nDVhn7p5u7FMcT2")
urls = ["https://storage.googleapis.com/hume-test-data/video/armisen-clip.mp4"]
configs = [ProsodyConfig(identify_speakers = True)]
job = client.submit_job(urls, configs)

print(job)
print("Running...")

job.await_complete()
job.download_predictions("predictions.json")
print("Predictions downloaded to predictions.json")

job.download_artifacts("artifacts.zip")
print("Artifacts downloaded to artifacts.zip")

with open('/Users/gill/Documents/GitHub/calhacks/backend/predictions.json', 'r') as f:
    # Load the contents of the file into a variable
    data = json.load(f)

# Print the contents of the variable
print(data)


Job(id="e6c3196b-9f03-48f5-9f00-2733b7a00068")
Running...
Predictions downloaded to predictions.json
Artifacts downloaded to artifacts.zip
[{'source': {'type': 'url', 'url': 'https://storage.googleapis.com/hume-test-data/video/armisen-clip.mp4'}, 'results': {'predictions': [{'file': 'armisen-clip.mp4', 'file_type': 'video_no_audio', 'models': {'prosody': {'metadata': {'confidence': 0.73583984, 'detected_language': 'en'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': 'A talk down any invitations', 'time': {'begin': 0.0, 'end': 2.18}, 'confidence': 0.70766604, 'speaker_confidence': None, 'emotions': [{'name': 'Admiration', 'score': 0.059178274124860764}, {'name': 'Adoration', 'score': 0.04490788280963898}, {'name': 'Aesthetic Appreciation', 'score': 0.023199710994958878}, {'name': 'Amusement', 'score': 0.3629228174686432}, {'name': 'Anger', 'score': 0.013151883147656918}, {'name': 'Anxiety', 'score': 0.013414605520665646}, {'name': 'Awe', 'score': 0.02520785480737686

In [15]:
# Extract predictions
predictions = data[0]['results']['predictions'][0]['models']['prosody']['grouped_predictions'][0]['predictions']

# Flatten to get just the predicted text
texts = []
for pred in predictions:
    texts.append(pred['text'])

df = pd.DataFrame({'text': texts})

print(df)

                          text
0  A talk down any invitations


In [16]:
flattened_data = []
for item in data:
    for prediction in item['results']['predictions'][0]['models']['prosody']['grouped_predictions']:
        for text_prediction in prediction['predictions']:
            for emotion in text_prediction['emotions']:
                flattened_data.append({
                    'Text': text_prediction['text'],
                    'Time_Begin': text_prediction['time']['begin'],
                    'Time_End': text_prediction['time']['end'],
                    'Emotion_Name': emotion['name'],
                    'Emotion_Score': emotion['score']
                })

# Create a Pandas DataFrame
df = pd.DataFrame(flattened_data)

# Display the DataFrame
print(df)

                           Text  Time_Begin  Time_End            Emotion_Name  \
0   A talk down any invitations         0.0      2.18              Admiration   
1   A talk down any invitations         0.0      2.18               Adoration   
2   A talk down any invitations         0.0      2.18  Aesthetic Appreciation   
3   A talk down any invitations         0.0      2.18               Amusement   
4   A talk down any invitations         0.0      2.18                   Anger   
5   A talk down any invitations         0.0      2.18                 Anxiety   
6   A talk down any invitations         0.0      2.18                     Awe   
7   A talk down any invitations         0.0      2.18             Awkwardness   
8   A talk down any invitations         0.0      2.18                 Boredom   
9   A talk down any invitations         0.0      2.18                Calmness   
10  A talk down any invitations         0.0      2.18           Concentration   
11  A talk down any invitati

In [17]:
grouped_df = df.groupby('Text')

# Display the grouped DataFrame
groups = []
texts = []
for name, group in grouped_df:
    print(f"Text: {name}")
    group = group.sort_values(by="Emotion_Score",ascending=False) 
    group = group[group["Emotion_Score"].apply(float) > .10]
    groups.append(group.reset_index(drop=True))
    texts.append(name)

Text: A talk down any invitations


In [18]:
send_data = {}
for i,text in enumerate(texts):
    df_json = groups[i].to_json()
    send_data[text] = json.loads(df_json)
json_data = json.dumps(send_data)


In [19]:
json_data

'{"A talk down any invitations": {"Text": {"0": "A talk down any invitations", "1": "A talk down any invitations", "2": "A talk down any invitations", "3": "A talk down any invitations", "4": "A talk down any invitations", "5": "A talk down any invitations", "6": "A talk down any invitations", "7": "A talk down any invitations", "8": "A talk down any invitations", "9": "A talk down any invitations"}, "Time_Begin": {"0": 0.0, "1": 0.0, "2": 0.0, "3": 0.0, "4": 0.0, "5": 0.0, "6": 0.0, "7": 0.0, "8": 0.0, "9": 0.0}, "Time_End": {"0": 2.18, "1": 2.18, "2": 2.18, "3": 2.18, "4": 2.18, "5": 2.18, "6": 2.18, "7": 2.18, "8": 2.18, "9": 2.18}, "Emotion_Name": {"0": "Excitement", "1": "Amusement", "2": "Joy", "3": "Determination", "4": "Interest", "5": "Satisfaction", "6": "Concentration", "7": "Pride", "8": "Triumph", "9": "Calmness"}, "Emotion_Score": {"0": 0.3735006452, "1": 0.3629228175, "2": 0.3458599448, "3": 0.2737347484, "4": 0.2713213563, "5": 0.2441013753, "6": 0.1496056914, "7": 0.11

In [20]:
newdict = {}
while texts != []:
    if len(texts) == 1:
        newdict[texts[0]] = groups[0]
        texts = []
    else:
        newdict[texts[0]] = groups[0]
        groups = groups[1:]
        texts = texts[1:]


In [30]:
newdict["A talk down any invitations"]
scoredict = {}

newdf = newdict["A talk down any invitations"].set_index('Emotion_Name')
newnewdf = newdf['Emotion_Score'].to_dict()


for key in newdict.keys():
    appendscore = {}
    newdicts = newdict[key].set_index("Emotion_Name")
    scoredict[key] = newdicts['Emotion_Score'].to_dict()



# for every element in the dictionary
    # we're going to make a dictionary that contains all emotion/emotion_score key/value pairs
    # append it to anotehr dicitonary that has key dick 



                                      Text  Time_Begin  Time_End  \
Emotion_Name                                                       
Excitement     A talk down any invitations         0.0      2.18   
Amusement      A talk down any invitations         0.0      2.18   
Joy            A talk down any invitations         0.0      2.18   
Determination  A talk down any invitations         0.0      2.18   
Interest       A talk down any invitations         0.0      2.18   
Satisfaction   A talk down any invitations         0.0      2.18   
Concentration  A talk down any invitations         0.0      2.18   
Pride          A talk down any invitations         0.0      2.18   
Triumph        A talk down any invitations         0.0      2.18   
Calmness       A talk down any invitations         0.0      2.18   

               Emotion_Score  
Emotion_Name                  
Excitement          0.373501  
Amusement           0.362923  
Joy                 0.345860  
Determination       0.273735

In [23]:
newdf = newdict["A talk down any invitations"].set_index('Emotion_Name')
newnewdf = newdf['Emotion_Score'].to_dict()

In [29]:
newnewdf

{'Excitement': 0.37350064516067505,
 'Amusement': 0.3629228174686432,
 'Joy': 0.34585994482040405,
 'Determination': 0.2737347483634949,
 'Interest': 0.2713213562965393,
 'Satisfaction': 0.2441013753414154,
 'Concentration': 0.14960569143295288,
 'Pride': 0.11558214575052261,
 'Triumph': 0.10451919585466385,
 'Calmness': 0.104410819709301}